
# Digging Analysis 

<a href="https://colab.research.google.com/github.com/healthonrails/annolid/blob/main/docs/tutorials/Digging_Analysis_tutorial.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, we'll show examples of how to perform digging behavior analysis.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Constant values

In [ ]:
has_gt = False # Set has_gt to True if you have it

In [ ]:
fps = 29.97
box_x1, box_y1, box_x2, box_y2 = 324, 178, 570, 344

# Util functions

In [ ]:
def convert_time_to_frame_number(time_stamp, fps=29.97):
    h, m, s = time_stamp.split(':')
    seconds, milliseconds = s.split('.')
    total_seconds = int(h) * 3600 + int(m) * 60 + int(seconds)
    total_frames = int(total_seconds * fps) + int(milliseconds) * fps // 1000
    return int(total_frames)


In [ ]:
def convert_frame_number_to_time(frame_number,fps=fps):
    total_seconds = frame_number / fps
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    milliseconds = int((total_seconds - int(total_seconds)) * 1000)
    time_stamp = f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
    return time_stamp


In [ ]:
import ast
from pycocotools import mask as mask_util
def mask_area(mask):
    """Calulate the area of a RLE mask.
    """
    try:
        area = mask_util.area(mask)
    except TypeError:
        mask = ast.literal_eval(mask)
        area = mask_util.area(mask)
    return area

# This section focuses on spiking neuron recordings. Skip it if it's not relevant to your project.






In [ ]:
spiking_csv = '/content/R2142_04-02-2022_nlx_mtx.csv'
df_spike = pd.read_csv(spiking_csv,header=None)

In [ ]:
df_spike.head()

In [ ]:
df_spike.describe()

In [ ]:
df_spike_time = df_spike[df_spike[3]>0][[0,3]]

In [ ]:
df_spike_time.head()

In [ ]:
def convert_seconds_to_frame_number(seconds, frame_rate=fps):
    frame_number = int(seconds * frame_rate)
    return frame_number

In [ ]:
df_spike_time['Frame_number'] = df_spike[0].apply(convert_seconds_to_frame_number)

In [ ]:
timestamps = df_spike_time.Frame_number.apply(convert_frame_number_to_time)

In [ ]:
df_spike_time['Timestamp'] = timestamps

In [ ]:
frame_number_mark_type = pd.Series(list(zip(df_spike_time.Frame_number,['event_end'] * len(df_spike_time.Frame_number) ))).astype('str')

In [ ]:
df_spike_time.head()

In [ ]:
df_spike_time['Frame_number'] = frame_number_mark_type

In [ ]:
df_spike_time.describe()

In [ ]:
df_spike_time[3].value_counts()

In [ ]:
df_19_05 = df_spike_time[df_spike_time[3]==19.05]

In [ ]:
df_17_04 = df_spike_time[df_spike_time[3]==17.04]

In [ ]:
df_17_03 = df_spike_time[df_spike_time[3]==17.03]

In [ ]:
df_19_06 = df_spike_time[df_spike_time[3]==19.06]

In [ ]:
df_20_14 = df_spike_time[df_spike_time[3]==20.14]

In [ ]:
df_20_14.head()

In [ ]:
# create the plot
fig, ax = plt.subplots(figsize=(16, 8))

# plot the ground truth if available
if has_gt:
    x_gt = df_gt.values
    y_gt = [ax.get_ylim()[1]] * len(x_gt) 
    ax.scatter(x_gt, y_gt, marker="|", color="red", s=150, label="Ground Truth")

# plot the predictions in separate rows
predictions = [
    (df_19_05["Timestamp"].apply(convert_time_to_frame_number).values, "19.5", "green"),
    (df_17_03["Timestamp"].apply(convert_time_to_frame_number).values, "17.3", "blue"),
    (df_17_04["Timestamp"].apply(convert_time_to_frame_number).values, "17.4", "orange"),
    (df_19_06["Timestamp"].apply(convert_time_to_frame_number).values, "19.6", "purple"),
    (df_20_14["Timestamp"].apply(convert_time_to_frame_number).values, "20.14", "brown")
]

# loop over the predictions and plot each set in a separate row
for i, (x_pred, label, color) in enumerate(predictions):
    y_pred = [ax.get_ylim()[1] + 2] * len(x_pred)
    ax.scatter(x_pred, y_pred, marker='|', color=color, s=150, label=label)

# add legend
#ax.legend(loc="lower right")

# show the plot
plt.show()

In [ ]:
df_spike_time.Frame_number.values

In [ ]:
df_spike_time_save = df_19_05[['Timestamp','Frame_number']]

In [ ]:
df_spike_time_save.dropna(inplace=True)

In [ ]:
df_spike_time_save.head()

In [ ]:
df_spike_time_save.to_csv("/content/timestamps_nlx_mtx.csv",index=False)

## If you need to compare the results, you should reload the ground truth timestamps.








In [ ]:
df_gt = pd.read_excel('/content/timestamp _2202.xlsx')

In [ ]:
df_gt.head()


In [ ]:
df_gt = df_gt['R_04-02-2022'].dropna().apply(convert_time_to_frame_number)
df_gt.values    

In [ ]:
len(df_gt.values)

## Load and process Annolid predict results

In [ ]:
annolid_results_csv = "/content/rats_v2_coco_dataset_R2202_12-21-2022_mask_rcnn_tracking_results_with_segmentation.csv" #@param
df_pred = pd.read_csv(annolid_results_csv)

In [ ]:
df_pred.head()

In [ ]:
df = df_pred

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

# Step 1: Filter the dataframe for the rat instances
threshold = 0.9  # Adjust the threshold as needed
filtered_df = df[(df['instance_name'] == 'rat') & (df['class_score'] > threshold)]
filtered_cup = df[(df['instance_name'] == 'cup') & (df['class_score'] > threshold)]

In [ ]:
# Step 2: Extract relevant columns for rat and cup positions
rat_positions = filtered_df[['frame_number', 'cx', 'cy']]
cup_positions = filtered_df[['frame_number', 'x1', 'y1', 'x2', 'y2']]


In [ ]:

# Step 3: Calculate distances between rat and cup for each frame
distances = []
for _, rat_row in rat_positions.iterrows():
    frame_number = rat_row['frame_number']
    rat_coords = (rat_row['cx'], rat_row['cy'])
    cup_coords = cup_positions[cup_positions['frame_number'] == frame_number][['x1', 'y1', 'x2', 'y2']].values
    if len(cup_coords) > 0:
        cup_center = ((cup_coords[:, 0] + cup_coords[:, 2]) / 2, (cup_coords[:, 1] + cup_coords[:, 3]) / 2)
        dist = np.min([distance.euclidean(rat_coords, center) for center in cup_center])
        distances.append((frame_number, dist))

In [ ]:
# Step 4: Apply time series analysis techniques
dist_df = pd.DataFrame(distances, columns=['frame_number', 'distance'])
dist_df['moving_average'] = dist_df['distance'].rolling(window=5, min_periods=1).mean()  # Moving average example


In [ ]:
# Step 5: Identify frames where rat is close to the cup
threshold_distance = 50  # Adjust the threshold as needed
close_frames = dist_df[dist_df['distance'] < threshold_distance]['frame_number']

# Print the close frames
print("Frame numbers where rat is close to the cup:")
print(close_frames)


In [ ]:
# Step 6: Calculate velocity and acceleration
frame_numbers = [frame_number for frame_number, _ in distances]
dist_values = [distance for _, distance in distances]

# Calculate time intervals between frames
time_intervals = np.diff(frame_numbers)

# Calculate velocity as change in distance per unit time
velocity = np.diff(dist_values) / time_intervals

# Calculate acceleration as change in velocity per unit time
acceleration = np.diff(velocity) / time_intervals[:-1]

# Print the velocity and acceleration
print("Velocity:", velocity)
print("Acceleration:", acceleration)

In [ ]:
# Thresholds for movement and reaching the cup
movement_threshold = -0.1  # Negative velocity threshold for movement
about_to_reach_threshold = 0.01  # Acceleration threshold for about to reach the cup

# Determine moments when the rat is moving towards the cup
moving_towards_cup_frames = [frame for frame, velocity in zip(frame_numbers[1:], velocity) if velocity < movement_threshold]

# Determine moments when the rat is about to reach the cup
about_to_reach_frames = [frame for frame, acceleration in zip(frame_numbers[2:], acceleration) if acceleration < about_to_reach_threshold]

# Print the frames when the rat is moving towards the cup and about to reach the cup
print("Frames when rat is moving towards the cup:", moving_towards_cup_frames)
print("Frames when rat is about to reach the cup:", about_to_reach_frames)


In [ ]:
# Thresholds
gap_threshold = 200  # Number of frames without a cup to consider as a gap
distance_threshold = 100  # Threshold distance for frame numbers in each group

# Step 7: Divide frames into periods with no cups for more than 100 frames
frame_gaps = []
current_gap = []
for frame_number in frame_numbers:
    if len(current_gap) == 0 or frame_number - current_gap[-1] == 1:
        current_gap.append(frame_number)
    else:
        if len(current_gap) > gap_threshold:
            frame_gaps.append(current_gap)
        current_gap = []
if len(current_gap) > gap_threshold:
    frame_gaps.append(current_gap)

# Step 8: Filter frame numbers in each group based on the distance threshold
filtered_frame_numbers = []
for gap in frame_gaps:
    gap_frame_numbers = [frame for frame in gap if frame in moving_towards_cup_frames]
    filtered_frame_numbers.extend(gap_frame_numbers)

# Print the filtered frame numbers
print("Filtered frame numbers:")
print(filtered_frame_numbers)


In [ ]:
df_cup = df_pred[df_pred['instance_name']=='cup']

In [ ]:
df_cup['mask_area'] = df_cup.segmentation.apply(mask_area)

In [ ]:
df_cup.head()

In [ ]:
df_cup[df_cup.frame_number == 29956]

In [ ]:
df_cup['mask_area'].describe()

In [ ]:
df_cup.class_score.describe()

In [ ]:
df_cup = df_cup[df_cup.mask_area > 700]

In [ ]:
df_cup = df_cup[df_cup.class_score > 0.99]

In [ ]:
df_cups = df_pred[df_pred.frame_number.isin(df_cup.frame_number)]

In [ ]:
df_cups_rats = df_cups[df_cups.frame_number.isin(df_cups[df_cups.instance_name == 'rat'].frame_number.unique())]

In [ ]:
df_cups_rats[df_cups_rats.frame_number == 38205]

In [ ]:
import pandas as pd

# Step 1: Filter dataframe to include frames with rat head and cup instances
rat_cup_df = df_cups_rats #df[(df['instance_name'] == 'rat head') | (df['instance_name'] == 'cup')]
# Step 2: Compute IoU for each frame
iou_list = []
for frame_num in rat_cup_df.frame_number:
    # Get binary masks for rat head and cup instances
    df_cur = rat_cup_df[rat_cup_df.frame_number == frame_num]
    rat_mask = df_cur[df_cur.instance_name == 'rat']['segmentation'].values[0]
    cup_mask = df_cur[df_cur.instance_name == 'cup']['segmentation'].values[0]
    rat_rle = ast.literal_eval(rat_mask)
    cup_rle = ast.literal_eval(cup_mask)
    # calculate the IoU between the rat and cup masks
    iou = mask_util.iou([rat_rle], [cup_rle], [0])[0][0]  # we assume there is only one rat and one cup mask
    iou_list.append((frame_num, iou))

# Convert results to dataframe
iou_df = pd.DataFrame(iou_list, columns=['frame_number', 'iou'])

In [ ]:
smoothed_iou = iou_df['iou'].rolling(15).mean()

In [ ]:
iou_df['smoothed_iou'] = smoothed_iou

In [ ]:
iou_df['diff_iou'] = iou_df.smoothed_iou.diff()


In [ ]:
# Compute rolling window of size 15 and count non-zero values
window_size = 15
nonzero_counts = iou_df['iou'].rolling(window_size).apply(lambda x: (x > 0).sum())


In [ ]:
iou_df['nonzero_counts'] = nonzero_counts

In [ ]:
iou_df.describe()

In [ ]:
prev_iou = 0.0
nonzeros_count_in_window = 1
res = set()
for idx, row in iou_df.iterrows():
    if row.nonzero_counts >= nonzeros_count_in_window and row.diff_iou > 0.000001 and  0 <= prev_iou <= 0.00000000000001:
        res.add(row.frame_number)
    prev_iou = row.diff_iou

In [ ]:
def reduce_array(arr,window_threshold = 500):
    result = [arr[0]]  # initialize result array with first element of input array
    for i in range(1, len(arr)):
        if arr[i] - result[-1] < window_threshold:
            result[-1] = min(result[-1], arr[i])
        else:
            result.append(arr[i])
    return result

In [ ]:
res = [int(m) for m in medians]

In [ ]:
preds = [int(fn) for fn in sorted(res)]


In [ ]:
res = reduce_array(preds,45)

In [ ]:
len(res)

In [ ]:
preds = res

In [ ]:
frames_cup_not_on_bottom = set(df_cup[(df_cup.cy < box_y1) | (df_cup.cy > box_y2) | (df_cup.cx < box_x1) | (df_cup.cx > box_x2)].frame_number.unique())


In [ ]:
preds = [pred for pred in preds if pred not in frames_cup_not_on_bottom]

In [ ]:
df_cup[df_cup.frame_number == 142073]

In [ ]:
preds = reduce_array(preds)

In [ ]:
len(preds), print(str(preds))

In [ ]:
time_stamps = []
for pred in preds:
    time_stamps.append(convert_frame_number_to_time(fps,pred))


In [ ]:
pred_s = pd.Series(time_stamps)

In [ ]:
pred_s_f = pd.DataFrame({'Timestamp':time_stamps,'Frame_number':preds})

In [ ]:
video_name = annolid_results_csv.split('dataset_')[-1].split('_mask')[0]

In [ ]:
timestamp_frame_number_csv = f'timestamps_frame_number_predicted_{video_name}.csv'

In [ ]:
pred_s_f.to_csv(timestamp_frame_number_csv, index=False)

In [ ]:
pred_timestamp_csv = f'predicted_{video_name}.csv'

In [ ]:
pred_s.to_csv(pred_timestamp_csv, index=False)

In [ ]:
# create the plot
fig, ax = plt.subplots(figsize=(16, 8))

# plot the data
#ax.plot(iou_df["frame_number"], iou_df["smoothed_iou"], label="Smoothed IoU")
#ax.plot(iou_df["frame_number"], iou_df["diff_iou"], label="Difference of IoU")

if has_gt:
    # plot the special marks
    x_gt = df_gt.values
    y_gt = [ax.get_ylim()[1]] * len(x_gt)
    ax.scatter(x_gt, y_gt, marker="|", color="red", s=150, label="Ground Truth")
x_pred = [int(eval(x)[0]) for x in df_spike_time_save["Frame_number"].values]
y_pred = [ax.get_ylim()[1]] * len(x_pred)
ax.scatter(x_pred, y_pred, marker='|',color='green', s=150, label="Predictions")

# add legend
ax.legend(loc="lower left")

# show the plot
plt.show()

# Download the timestamp results

In [ ]:
from google.colab.files import download

In [ ]:
download(pred_timestamp_csv)
download(timestamp_frame_number_csv)